# Prepare

In [1]:
# import necessaries
import tushare as ts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as ss
import tushare as ts
pro = ts.pro_api()
pd.set_option('display.max_rows', 1000)
# set plot conf
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# ignore warning
import warnings
warnings.filterwarnings("ignore")

# Get option info

In [ ]:
option_info = pro.opt_basic(
    exchange='SSE',
    call_put='C',
    fields='ts_code,name,exercise_price, list_date,delist_date,maturity_date')
for i in {'maturity_date', 'list_date', 'delist_date'}:
    option_info[i] = pd.to_datetime(option_info[i])
option_info

In [ ]:
option_info[option_info['name'].str.contains('(1703..2.055|1706..2.153|1709..2.25|1712..2.16)')]

In [ ]:
def get_option_info(key_words_list):
    temp = option_info.copy()
    for i in key_words_list:
        temp = temp[temp['name'].str.contains(i)]
    return temp
get_option_info(['1703','2.055'])

## take an example

---- 2017 03 option

In [ ]:
LIST_03 = ['1703','2.055']

In [ ]:
def get_opiton_code(key_words_list):
    OPTION_CODE = str(get_option_info(key_words_list)['ts_code'].values[0])
    return OPTION_CODE
get_opiton_code(['1703','2.055'])

In [ ]:
FROM_DATE = '20160728'
FROM_DATE_FORE = '20160428'
END_DATE = '20170322'
VOL_PERIODS = 30

In [ ]:
def get_date_info(key_words_list):

    return get_option_info(get_opiton_code(key_words_list))['list_date']

    return BEGIN_DATE, END_DATE
    
get_starting_date(list_03)

# Get option trade data

In [ ]:
def col_to_date_and_as_index(df, col_name):
    df[col_name] = pd.to_datetime(df[col_name])
    df.index = df[col_name]

In [ ]:
def get_option_trade_data(option_code):
    pro = ts.pro_api()
    option_trade = pro.opt_daily(ts_code=option_code)
    col_to_date_and_as_index(option_trade, 'trade_date')
    return option_trade

opt_3 = get_option_trade_data("10000671.SH")
opt_3

In [ ]:
def draw_pic(data, title_name):
    plt.figure(figsize=(12,8))
    plt.plot(data)
    plt.title(title_name)
    plt.show()

draw_pic(opt_3.close, 'opt_3')

# Construct BS model

In [ ]:

def d1(S0, K, r, sigma, T):
    return (np.log(S0 / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))


def d2(S0, K, r, sigma, T):
    return (np.log(S0 / K) + (r - sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))


def BlackScholes(type, S0, K, r, sigma, T):
    """
    S0  stock price
    K   strike price
    r   continuously compounded risk-free rate:
    sigma   volatility of the stock price per year
    T   time to maturity in trading years

    """
    if type == "C":
        return S0 * ss.norm.cdf(d1(S0, K, r, sigma, T)) - K * np.exp(-r * T) * ss.norm.cdf(d2(S0, K, r, sigma, T))
    else:
        return K * np.exp(-r * T) * ss.norm.cdf(-d2(S0, K, r, sigma, T)) - S0 * ss.norm.cdf(-d1(S0, K, r, sigma, T))


In [ ]:
FROM_DATE = '20160728'
FROM_DATE_FORE = '20160428'
END_DATE = '20170322'
VOL_PERIODS = 30
OPTION_CODE = '10000671.SH'

## get ETF50 trade data

In [ ]:
def get_underlying_asset_trade_data():
    #pro = ts.pro_api()
    data = pro.fund_daily(ts_code='510050.SH', start_date=FROM_DATE_FORE, end_date=END_DATE)
    data['log_return'] = np.log(data['close']) - np.log(data['close']).shift(1)
    data['std'] = data['log_return'].rolling(window=VOL_PERIODS).std(ddof=1) * np.sqrt(250 / VOL_PERIODS)
    col_to_date_and_as_index(data, 'trade_date')
    return data

stock_trade = get_stock_trade_data()
stock_trade

In [ ]:
def draw_pic(data, title_name):
    plt.figure(figsize=(12,8))
    plt.plot(data)
    plt.title(title_name)
    plt.show()

draw_pic(stock_trade.close, 'ETF50')

# get option info

In [ ]:
def get_option_trade_data(option_code):
    pro = ts.pro_api()
    option_trade = pro.opt_daily(ts_code=option_code)
    col_to_date_and_as_index(option_trade, 'trade_date')
    return option_trade

option_info = get_option_info_data(OPTION_CODE)
option_info

In [ ]:
FROM_DATE = '20160728'
FROM_DATE_FORE = '20160428'
END_DATE = '20170322'
K = option_info['exercise_price'].values[0]

# TODO: 修改为自动

# get option trade data

In [ ]:
def get_option_trade_data(option_code):
    pro = ts.pro_api()
    option_trade = pro.opt_daily(ts_code=option_code)
    col_to_date_and_as_index(option_trade, 'trade_date')
    return option_trade

option_trade = get_option_trade_data(OPTION_CODE)
option_trade

In [ ]:
draw_pic(option_trade.close, 'option')

In [ ]:
def get_stock_trade_data():
    pro = ts.pro_api()
    data = pro.index_daily(ts_code='000016.SH', start_date=FROM_DATE_FORE, end_date=END_DATE)
    data['log_return'] = np.log(data['close']) - np.log(data['close']).shift(1)
    data['std'] = data['log_return'].rolling(window=VOL_PERIODS).std(ddof=1) * np.sqrt(250 / VOL_PERIODS)
    col_to_date_and_as_index(data, 'trade_date')
    return data

SSE_50 = get_stock_trade_data()
SSE_50

In [ ]:
plt.figure(figsize=(12,8))
#plt.plot(option_trade.close, label='option')
#plt.plot(stock_trade.close, label='stock')
plt.plot(SSE_50.close, label='SSE50')

plt.legend(loc='best')

plt.show()

In [ ]:
 option_trade = get_option_trade_data(OPTION_CODE)
    rate = get_rate_data()
    option_trade['T'] = (option_info['maturity_date'].values - option_trade['trade_date']).dt.days / 365.0
    K = option_info['exercise_price'].values[0]
    summary = pd.DataFrame(index=option_trade.index)
    summary['S'] = stock_trade['close']
    summary['T'] = (option_info['maturity_date'].values - option_trade['trade_date']).dt.days
    summary['r'] = rate['3m']
    summary['sigma'] = stock_trade['std']
    summary = summary.dropna()
    summary['bs_simulate'] = BlackScholes('C', summary['S'], K, summary['r'], summary['sigma'], summary['T'])
    summary['real_price'] = option_trade['close']
    simulate_vs_real = summary[['bs_simulate','real_price']]
    simulate_vs_real.plot()
    plt.show()

In [ ]:
pro = ts.pro_api()
data = pro.fund_daily(ts_code='510050.SH')
# data['log_return'] = np.log(data['close']) - np.log(data['close']).shift(1)
# data['std'] = data['log_return'].rolling(window=VOL_PERIODS).std(ddof=1) * np.sqrt(250 / VOL_PERIODS)
col_to_date_and_as_index(data, 'trade_date')

In [ ]:
data

In [ ]:
draw_pic(data.close, 'option')

In [ ]:
max(data.close)

In [ ]:
pro = ts.pro_api()
option_trade = pro.opt_daily(ts_code = '10001473.SH')
col_to_date_and_as_index(option_trade, 'trade_date')
option_trade

In [ ]:
option_info = pro.opt_basic(exchange='SSE', call_put = 'C',
              fields='ts_code,name,exercise_price,last_edate, list_date,delist_date')
option_info

In [ ]:
option_info[option_info['name'].str.contains('(1811..2.50|1811..2.55)')]

### suspend

In [10]:
option_info_all = pro.opt_basic(
    exchange='SSE',
    call_put='C',
    fields='ts_code,name,exercise_price, list_date,delist_date,maturity_date')
for i in {'maturity_date', 'list_date', 'delist_date'}:
    option_info_all[i] = pd.to_datetime(option_info[i])
option_info_all

,ts_code,name,exercise_price,maturity_date,list_date,delist_date
0,10000869.SH,华夏上证50ETF期权1705认购2.25,2.250,2017-05-24,2017-03-23,2017-05-24
1,10000870.SH,华夏上证50ETF期权1705认购2.30,2.300,2017-05-24,2017-03-23,2017-05-24
2,10000871.SH,华夏上证50ETF期权1705认购2.35,2.350,2017-05-24,2017-03-23,2017-05-24
3,10000872.SH,华夏上证50ETF期权1705认购2.40,2.400,2017-05-24,2017-03-23,2017-05-24
4,10000873.SH,华夏上证50ETF期权1705认购2.45,2.450,2017-05-24,2017-03-23,2017-05-24
5,10000879.SH,华夏上证50ETF期权1705认购2.50,2.500,2017-05-24,2017-04-06,2017-05-24
6,10001253.SH,华夏上证50ETF期权1804认购2.95,2.950,2018-04-25,2018-03-01,2018-04-25
7,10001254.SH,华夏上证50ETF期权1804认购3.00,3.000,2018-04-25,2018-03-01,2018-04-25
8,10001255.SH,华夏上证50ETF期权1804认购3.10,3.100,2018-04-25,2018-03-01,2018-04-25
9,10001179.SH,华夏上证50ETF期权1801认购3.30,3.300,2018-01-24,2018-01-08,2018-01-24


### Function

In [11]:
def etf_close(data, from_date, end_date):
    return data['close'][from_date: end_date]


def get_opiton_code(key_words_list):
    OPTION_CODE = str(get_option_info(key_words_list)['ts_code'].values[0])
    return OPTION_CODE


def get_option_info(key_words_list):
    temp = option_info_all.copy()
    for i in key_words_list:
        temp = temp[temp['name'].str.contains(i)]
    return temp


def plot_close_data(from_date, end_date):
    df = etf_close(from_date, end_date)
    df.plot()
    plt.show()


def get_stock_trade_data():
    pro = ts.pro_api()
    data = pro.fund_daily(ts_code='510050.SH', start_date=FROM_DATE_FORE, end_date=END_DATE)
    data['log_return'] = np.log(data['close']) - np.log(data['close']).shift(1)
    data['std'] = data['log_return'].rolling(window=VOL_PERIODS).std(ddof=1) * np.sqrt(250 / VOL_PERIODS)
    col_to_date_and_as_index(data, 'trade_date')
    return data


def col_to_date_and_as_index(df, col_name):
    df[col_name] = pd.to_datetime(df[col_name])
    df.index = df[col_name]


def get_option_info_data(option_code):
    pro = ts.pro_api()
    option_info = pro.opt_basic(exchange='SSE',
                                fields='ts_code,name,exercise_price, maturity_date, list_date, delist_date')
    for i in {'maturity_date', 'list_date', 'delist_date'}:
        option_info[i] = pd.to_datetime(option_info[i])
    return option_info[option_info.ts_code == option_code]


def get_option_trade_data(option_code):
    pro = ts.pro_api()
    option_trade = pro.opt_daily(ts_code=option_code)
    col_to_date_and_as_index(option_trade, 'trade_date')
    return option_trade


def get_rate_data():
    pro = ts.pro_api()
    df = pro.shibor(start_date=FROM_DATE, end_date=END_DATE, fields='date,3m')
    df['3m'] = np.log(df['3m'])
    col_to_date_and_as_index(df, 'date')
    return df

In [12]:
LIST_03 = ['1703','2.055']
OPTION_CODE = get_opiton_code(LIST_03)
OPTION_INFO = get_option_info_data(OPTION_CODE)

# stock_trade = get_stock_trade_data()

In [13]:
OPTION_CODE

'10000671.SH'

In [14]:
OPTION_INFO

,ts_code,name,exercise_price,maturity_date,list_date,delist_date
921,10000671.SH,华夏上证50ETF期权1703认购2.055,2.055,2017-03-22,2016-07-28,2017-03-22


In [ ]:
'def:
    # pro = ts.pro_api()  # initialize

    stock_trade = get_stock_trade_data()
    option_info = get_option_info_data(OPTION_CODE)
    option_trade = get_option_trade_data(OPTION_CODE)
    rate = get_rate_data()
    option_trade['T'] = (option_info['maturity_date'].values - option_trade['trade_date']).dt.days / 365.0
    K = option_info['exercise_price'].values[0]
    summary = pd.DataFrame(index=option_trade.index)
    summary['S'] = stock_trade['close']
    summary['T'] = (option_info['maturity_date'].values - option_trade['trade_date']).dt.days
    summary['r'] = rate['3m']
    summary['sigma'] = stock_trade['std']
    summary = summary.dropna()
    summary['bs_simulate'] = BlackScholes('C', summary['S'], K, summary['r'], summary['sigma'], summary['T'])
    summary['real_price'] = option_trade['close']
    simulate_vs_real = summary[['bs_simulate','real_price']]
    simulate_vs_real.plot()
    plt.show()